# GDAL day 2

GDAL - Geospatial Data Abstration Library

#### Set up
Use - `opengeo` micromamba env. 
Move to `gdal-tools` folder.

In [2]:
cd /Users/abharathi/Documents/gis_data/gdal-tools/

/Users/abharathi/Documents/gis_data/gdal-tools


In [3]:
ls

1870_southern-india.jpg*      landsat8/
5d16a93f1cf0f6000579ad2c.tif  london_1m_dsm/
batch.py*                     naip/
batch_parallel.py*            precipitation.gpkg
earth_at_night.jpg*           prism/
earthquakes/                  spatial_query.gpkg
gdal_stuff.qgz                srtm/
geonames/                     worldcities.csv*


## Talk to Cloud systems via streaming

In [1]:
!gdalinfo /vsigs/spatialthoughts-public-data/viirs_ntl_2021_global.tif --config GS_NO_SIGN_REQUEST YES

Driver: GTiff/GeoTIFF
Files: /vsigs/spatialthoughts-public-data/viirs_ntl_2021_global.tif
Size is 80152, 80196
Coordinate System is:
PROJCRS["WGS 84 / Pseudo-Mercator",
    BASEGEOGCRS["WGS 84",
        ENSEMBLE["World Geodetic System 1984 ensemble",
            MEMBER["World Geodetic System 1984 (Transit)"],
            MEMBER["World Geodetic System 1984 (G730)"],
            MEMBER["World Geodetic System 1984 (G873)"],
            MEMBER["World Geodetic System 1984 (G1150)"],
            MEMBER["World Geodetic System 1984 (G1674)"],
            MEMBER["World Geodetic System 1984 (G1762)"],
            MEMBER["World Geodetic System 1984 (G2139)"],
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]],
            ENSEMBLEACCURACY[2.0]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["Popular Visualisation Pseudo-Mercator",
        METHOD["Popular Visualisation Pseudo Mer

## Working with aerial imagery

In [34]:
cd ../naip

/Users/abharathi/Documents/gis_data/gdal-tools/naip


In [35]:
ls

aoi.cpg*        output_2_1.jp2* output_3_8.jp2* output_5_7.jp2* output_7_6.jp2*
aoi.dbf*        output_2_2.jp2* output_4_1.jp2* output_5_8.jp2* output_7_7.jp2*
aoi.prj*        output_2_3.jp2* output_4_2.jp2* output_6_1.jp2* output_7_8.jp2*
aoi.qpj*        output_2_4.jp2* output_4_3.jp2* output_6_2.jp2* output_8_1.jp2*
aoi.shp*        output_2_5.jp2* output_4_4.jp2* output_6_3.jp2* output_8_2.jp2*
aoi.shx*        output_2_6.jp2* output_4_5.jp2* output_6_4.jp2* output_8_3.jp2*
filelist.txt    output_2_7.jp2* output_4_6.jp2* output_6_5.jp2* output_8_4.jp2*
output_1_1.jp2* output_2_8.jp2* output_4_7.jp2* output_6_6.jp2* output_8_5.jp2*
output_1_2.jp2* output_3_1.jp2* output_4_8.jp2* output_6_7.jp2* output_8_6.jp2*
output_1_3.jp2* output_3_2.jp2* output_5_1.jp2* output_6_8.jp2* output_8_7.jp2*
output_1_4.jp2* output_3_3.jp2* output_5_2.jp2* output_7_1.jp2* output_8_8.jp2*
output_1_5.jp2* output_3_4.jp2* output_5_3.jp2* output_7_2.jp2*
output_1_6.jp2* output_3_5.jp2* output_5_4.jp2* output_7

### Mosaicing aerial images

In [6]:
ls *.jp2 > filelist.txt

**Alpha** band is an additional band that simply states which cells have value and which have no-Data. It is an image mask. Alpha is useful when you compress raster using PNG or JP2 formats and reduce quantization to 8bit Uint8. With such low dynamic range, it becomes tricky to use `0` or `255` as no-data.

In [10]:
!gdalinfo output_1_1.jp2

Driver: JP2OpenJPEG/JPEG-2000 driver based on OpenJPEG library
Files: output_1_1.jp2
Size is 5000, 5000
Coordinate System is:
PROJCRS["NAD83 / UTM zone 10N",
    BASEGEOGCRS["NAD83",
        DATUM["North American Datum 1983",
            ELLIPSOID["GRS 1980",6378137,298.257222101,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4269]],
    CONVERSION["UTM zone 10N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-123,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
            LENGTHUNIT["metre",1],
 

#### Build virtual tile that mosaics the drone images using `gdalbuildvrt`

In [36]:
%time
!gdalbuildvrt -input_file_list filelist.txt naip.vrt

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 12.9 µs
0...10...20...30...40...50...60...70...80...90...100 - done.


#### Create overview file for visualizing `gdal_translate`

In [38]:
%%time
!gdal_translate -of JPEG -outsize 2% 2% naip.vrt naip_preview.jpg

Input file size is 40000, 40000
0...10...20...30...40...50...60...70...80...90...100 - done.
CPU times: user 38.4 ms, sys: 24.2 ms, total: 62.6 ms
Wall time: 3.22 s


<img src="Users/abharathi/Documents/gis_data/gdal-tools/naip/naip_preview.jpg">

In [39]:
%%time
!gdaltindex -write_absolute_path index.json --optfile filelist.txt

Creating new index file...
CPU times: user 20 ms, sys: 15.3 ms, total: 35.4 ms
Wall time: 1.17 s


## Processing satellite image

In [45]:
cd ../landsat8/

/Users/abharathi/Documents/gis_data/gdal-tools/landsat8


In [46]:
ls -lh

total 1845384
-rwxr-xr-x@ 1 abharathi  staff   110M Oct 12  2019 RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B2.tif*
-rwxr-xr-x@ 1 abharathi  staff   115M Oct 12  2019 RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B3.tif*
-rwxr-xr-x@ 1 abharathi  staff   115M Oct 12  2019 RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B4.tif*
-rwxr-xr-x@ 1 abharathi  staff   131M Oct 12  2019 RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B5.tif*
-rwxr-xr-x@ 1 abharathi  staff   430M Oct 12  2019 RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B8.tif*
-rw-r--r--  1 abharathi  staff   2.5K Apr 26 06:47 rgb.vrt


#### Use `gdalbuildvrt` to composite bands into a virtual raster

In [30]:
!gdalbuildvrt -o rgb.vrt -separate \
  RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B4.tif \
  RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B3.tif \
  RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B2.tif

0...10...20...30...40...50...60...70...80...90...100 - done.


In [47]:
!gdalbuildvrt -o allbands.vrt -separate \
  RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B4.tif \
  RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B3.tif \
  RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B2.tif \
  RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B5.tif \
  RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B8.tif

0...10...20...30...40...50...60...70...80...90...100 - done.


#### Use `gdaltranslate` to convert virtual to persisted file

In [41]:
%%time
!gdal_translate rgp.vrt rgb.tif -co PHOTOMETRIC=RGB -co COMPRESS=DEFLATE

ERROR 4: rgp.vrt: No such file or directory
CPU times: user 7.96 ms, sys: 9.33 ms, total: 17.3 ms
Wall time: 631 ms


#### Build a visualization product that applies histogram stretch

In [50]:
%%time
!gdal_translate -scale 0 0.3 0 255 -ot Byte rgb.vrt rgb_stretch.tif

Input file size is 7701, 7861
Warning 1: for band 1, nodata value has been clamped to 0, the original value being out of range.
Warning 1: for band 2, nodata value has been clamped to 0, the original value being out of range.
Warning 1: for band 3, nodata value has been clamped to 0, the original value being out of range.
0...10...20...30...40...50...60...70...80...90...100 - done.
CPU times: user 41.6 ms, sys: 21.8 ms, total: 63.4 ms
Wall time: 4.17 s


## Raster algebra using `gdal_calc.py`

In [52]:
!which gdal_calc.py

/Users/abharathi/micromamba/envs/opengeo/bin/gdal_calc.py


In [66]:
%run -i /Users/abharathi/micromamba/envs/opengeo/bin/gdal_calc.py

usage: gdal_calc.py [--help] --calc [expression ...] [-a [filename ...]]
                    [--a_band [n ...]] [-b [filename ...]] [--b_band [n ...]]
                    [-c [filename ...]] [--c_band [n ...]] [-d [filename ...]]
                    [--d_band [n ...]] [-e [filename ...]] [--e_band [n ...]]
                    [-f [filename ...]] [--f_band [n ...]] [-g [filename ...]]
                    [--g_band [n ...]] [-h [filename ...]] [--h_band [n ...]]
                    [-i [filename ...]] [--i_band [n ...]] [-j [filename ...]]
                    [--j_band [n ...]] [-k [filename ...]] [--k_band [n ...]]
                    [-l [filename ...]] [--l_band [n ...]] [-m [filename ...]]
                    [--m_band [n ...]] [-n [filename ...]] [--n_band [n ...]]
                    [-o [filename ...]] [--o_band [n ...]] [-p [filename ...]]
                    [--p_band [n ...]] [-q [filename ...]] [--q_band [n ...]]
                    [-r [filename ...]] [--r_band [n ...]] [-s [

SystemExit: 2

#### Performing NDVI

In [ ]:
%%time
%run -i /Users/abharathi/micromamba/envs/opengeo/bin/gdal_calc.py \
  -A RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B5.TIF \
  -B RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B4.TIF \
  --outfile ndvi.tif --calc="(A-B)/(A+B)" --NoDataValue=-999

### Exercise - creating FCC viz product

In [69]:
%%time
!gdalbuildvrt -o nrg.vrt -separate \
RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B5.tif \
RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B4.tif \
RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B3.tif

0...10...20...30...40...50...60...70...80...90...100 - done.
CPU times: user 5.41 ms, sys: 9.25 ms, total: 14.7 ms
Wall time: 424 ms


In [73]:
%%time
!gdal_translate -of JPEG nrg.vrt nrg.jpg -co COMPRESS=JPG -scale 0 0.3 0 255 -ot Byte -outsize 10% 10%

Input file size is 7701, 7861
Warning 1: for band 1, nodata value has been clamped to 0, the original value being out of range.
Warning 1: for band 2, nodata value has been clamped to 0, the original value being out of range.
Warning 1: for band 3, nodata value has been clamped to 0, the original value being out of range.
Warning 6: driver JPEG does not support creation option COMPRESS
0...10...20...30...40...50...60...70...80...90...100 - done.
CPU times: user 9.38 ms, sys: 11.1 ms, total: 20.4 ms
Wall time: 589 ms


![](nrg.jpg)

### Pansharpen

In [76]:
!which gdal_pansharpen.py

/Users/abharathi/micromamba/envs/opengeo/bin/gdal_pansharpen.py


In [78]:
%%time
%run -i /Users/abharathi/micromamba/envs/opengeo/bin/gdal_pansharpen.py \
RT_LC08_L1TP_137042_20190920_20190926_01_T1_2019-09-20_B8.tif \
rgb.vrt pansharpened.vrt -r bilinear -co COMPRESS=DEFLATE -co PHOTOMETRIC=RGB

CPU times: user 6.39 ms, sys: 9.63 ms, total: 16 ms
Wall time: 68.6 ms


In [80]:
%%time
!gdal_translate -of JPEG pansharpened.vrt pansharpened.jpg -scale 0 0.3 0 255 -ot Byte -outsize 40% 40%

Input file size is 15401, 15721
Warning 1: for band 1, nodata value has been clamped to 0, the original value being out of range.
Warning 1: for band 2, nodata value has been clamped to 0, the original value being out of range.
Warning 1: for band 3, nodata value has been clamped to 0, the original value being out of range.
0...10...20...30...40...50...60...70...80...90...100 - done.
CPU times: user 97.1 ms, sys: 40.8 ms, total: 138 ms
Wall time: 12.8 s


## Exercise - Lidar of UK viewshed analysis

STEPS
1. mosaic using buildvrt
2. assign SRS of 4326 and persist raster
3. reproject using gdaltransform
4. gdal_viewshed to produce the viewshed raster

In [83]:
cd ../london_1m_dsm/

/Users/abharathi/Documents/gis_data/gdal-tools/london_1m_dsm


In [84]:
ls *.asc > filelist.txt

In [85]:
%%time
!gdalbuildvrt  -input_file_list filelist.txt londondsm.vrt

0...10...20...30...40...50...60...70...80...90...100 - done.
CPU times: user 25.1 ms, sys: 17.7 ms, total: 42.8 ms
Wall time: 1.55 s


In [86]:
# No CRS in the vrt
!gdalinfo londondsm.vrt

Driver: VRT/Virtual Raster
Files: londondsm.vrt
       tq3080_DSM_1M.asc
       tq3081_DSM_1M.asc
       tq3082_DSM_1M.asc
       tq3083_DSM_1M.asc
       tq3084_DSM_1M.asc
       tq3180_DSM_1M.asc
       tq3181_DSM_1M.asc
       tq3182_DSM_1M.asc
       tq3183_DSM_1M.asc
       tq3184_DSM_1M.asc
       tq3280_DSM_1M.asc
       tq3281_DSM_1M.asc
       tq3282_DSM_1M.asc
       tq3283_DSM_1M.asc
       tq3284_DSM_1M.asc
       tq3380_DSM_1M.asc
       tq3381_DSM_1M.asc
       tq3382_DSM_1M.asc
       tq3383_DSM_1M.asc
       tq3384_DSM_1M.asc
       tq3480_DSM_1M.asc
       tq3481_DSM_1M.asc
       tq3482_DSM_1M.asc
       tq3483_DSM_1M.asc
       tq3484_DSM_1M.asc
Size is 5000, 5000
Origin = (530000.000000000000000,185000.000000000000000)
Pixel Size = (1.000000000000000,-1.000000000000000)
Corner Coordinates:
Upper Left  (  530000.000,  185000.000) 
Lower Left  (  530000.000,  180000.000) 
Upper Right (  535000.000,  185000.000) 
Lower Right (  535000.000,  180000.000) 
Center      (  

In [89]:
%%time
#Assign SRS
!gdal_translate londondsm.vrt londondsm.tif \
  -co COMPRESS=DEFLATE -co PREDICTOR=2 \
  -a_srs EPSG:27700

Input file size is 5000, 5000
0...10...20...30...40...50...60...70...80...90...100 - done.
CPU times: user 30.6 ms, sys: 18.2 ms, total: 48.8 ms
Wall time: 3.3 s


### Reproject

In [ ]:
!gdaltransform -s_srs EPSG:4326